# A comparison of CAO points between 2019, 2020, and 2021.
***

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

- How to load CAO points information from the CAO website.
- A detailed comparison of CAO points in 2019, 2020, and 2021 using pandas

# References
[1]

***
# End